In [2]:
throwaway_df = sqlContext.createDataFrame([('Anthony', 10), ('Julia', 20), ('Fred', 5)], ('name', 'count'))
throwaway_df.select(throwaway_df['count']).show()

+-----+
|count|
+-----+
|   10|
|   20|
|    5|
+-----+



In [3]:
import re
import datetime

#Quick test of regular expression library
m = re.search('(?<=abc)def', 'abcdef')
print(m.group(0))

#Quick test of the datetime library
print('This was last run on: {0}'.format(datetime.datetime.now()))

def
This was last run on: 2017-04-27 14:17:58.795141


In [4]:
import sys
import os
log_file_path = 'hdfs:/' + os.path.join('user', 'root', 'NASA_access_log_Jul95')
base_df = sqlContext.read.text(log_file_path)
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
base_df.count()

1891715

In [6]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size')).dropna(subset='status')
split_df.show(truncate=False)

+--------------------------+--------------------------+-------------------------------------------------+------+------------+
|host                      |timestamp                 |path                                             |status|content_size|
+--------------------------+--------------------------+-------------------------------------------------+------+------------+
|199.72.81.55              |01/Jul/1995:00:00:01 -0400|/history/apollo/                                 |200   |6245        |
|unicomp6.unicomp.net      |01/Jul/1995:00:00:06 -0400|/shuttle/countdown/                              |200   |3985        |
|199.120.110.21            |01/Jul/1995:00:00:09 -0400|/shuttle/missions/sts-73/mission-sts-73.html     |200   |4085        |
|burger.letters.com        |01/Jul/1995:00:00:11 -0400|/shuttle/countdown/liftoff.html                  |304   |0           |
|199.120.110.21            |01/Jul/1995:00:00:11 -0400|/shuttle/missions/sts-73/sts-73-patch-small.gif  |200   |4179  

In [7]:
base_df.filter(base_df['value'].isNull()).count()

0

In [8]:
bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                              split_df['content_size'].isNull()
                              )
bad_rows_df.count()

19726

In [9]:
from pyspark.sql.functions import col, sum
def count_null(col_name):
    return sum(col(col_name).isNull().cast('integer')).alias(col_name)
exprs = [count_null(col_name) for col_name in split_df.columns]
split_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|       19726|
+----+---------+----+------+------------+



In [10]:
from pyspark.sql.functions import lit, concat
bad_content_size_df = base_df.filter(~ base_df['value'].rlike(r'\d+$'))
bad_content_size_df.count()

19727

In [11]:
cleaned_df = split_df.na.fill({'content_size': 0})
exprs = [count_null(col_name) for col_name in cleaned_df.columns]
cleaned_df.agg(*exprs).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     0|           0|
+----+---------+----+------+------------+



In [12]:
from pyspark.sql.functions import udf

month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    return "{0:02d}-{1:02d}-{2:04d} {3:02d}:{4:02d}:{5:02d}".format(
        int(s[7:11]),
        month_map[s[3:6]],
        int(s[0:2]),
        int(s[12:14]),
        int(s[15:17]),
        int(s[18:20])
    )

u_parse_time = udf(parse_clf_time)
logs_df = cleaned_df.select('*', u_parse_time(split_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

logs_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- path: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- content_size: integer (nullable = false)
 |-- time: timestamp (nullable = true)



In [13]:
logs_df.show(truncate=True)
print(total_log_entries)
#logs_df.cache()

+--------------------+--------------------+------+------------+--------------------+
|                host|                path|status|content_size|                time|
+--------------------+--------------------+------+------------+--------------------+
|       199.72.81.55 |    /history/apollo/|   200|        6245|1995-07-01 00:00:...|
|unicomp6.unicomp....| /shuttle/countdown/|   200|        3985|1995-07-01 00:00:...|
|     199.120.110.21 |/shuttle/missions...|   200|        4085|1995-07-01 00:00:...|
| burger.letters.com |/shuttle/countdow...|   304|           0|1995-07-01 00:00:...|
|     199.120.110.21 |/shuttle/missions...|   200|        4179|1995-07-01 00:00:...|
| burger.letters.com |/images/NASA-logo...|   304|           0|1995-07-01 00:00:...|
| burger.letters.com |/shuttle/countdow...|   200|           0|1995-07-01 00:00:...|
|    205.212.115.106 |/shuttle/countdow...|   200|        3985|1995-07-01 00:00:...|
|        d104.aa.net | /shuttle/countdown/|   200|        3985|19

In [14]:
content_size_summary_df = logs_df.describe(['content_size'])
content_size_summary_df.show()

+-------+------------------+
|summary|      content_size|
+-------+------------------+
|  count|           1891714|
|   mean|20455.509390425825|
| stddev|  76957.4154501093|
|    min|                 0|
|    max|           6823936|
+-------+------------------+



In [15]:
from pyspark.sql import functions as sqlFunctions
content_size_stats = (logs_df
                      .agg(sqlFunctions.min(logs_df['content_size']),
                           sqlFunctions.avg(logs_df['content_size']),
                           sqlFunctions.max(logs_df['content_size']))
                      .first())

print('Using SQL Functions: ')
print('Content Size Avg: {1:,.2f}; Min: {0:,.2f}; Max: {2:,.0f}'.format(*content_size_stats))

Using SQL Functions: 
Content Size Avg: 20,455.51; Min: 0.00; Max: 6,823,936


In [16]:
status_to_count_df = (logs_df
                      .groupBy('status')
                      .count()
                      .sort('status')
                      .cache())
status_to_count_length = status_to_count_df.count()
print('Found %d response codes'%status_to_count_length)
status_to_count_df.show()

Found 8 response codes
+------+-------+
|status|  count|
+------+-------+
|   200|1701534|
|   302|  46573|
|   304| 132627|
|   400|      5|
|   403|     54|
|   404|  10845|
|   500|     62|
|   501|     14|
+------+-------+



In [17]:
host_sum_df = (logs_df
               .groupBy('host')
               .count())
host_more_than_10_df = (host_sum_df
                        .filter(host_sum_df['count']>10)
                        .select(host_sum_df['host']))
host_more_than_10_df.show(truncate=False)

+----------------------------+
|host                        |
+----------------------------+
|picard.microsys.net         |
|ts900-418.singnet.com.sg    |
|winnie.freenet.mb.ca        |
|alpha2.csd.uwm.edu          |
|ivy-b0.aip.realtime.net     |
|maria-4e.ip.realtime.net    |
|kauai-9.u.aloha.net         |
|edinburgh3.easynet.co.uk    |
|192.127.29.43               |
|st2158.infonet.tufts.edu    |
|moe06.slip.yorku.ca         |
|lcy-ip19.halcyon.com        |
|pm1-orl7.iag.net            |
|dd08-044.compuserve.com     |
|146.154.26.105              |
|nrba1_onlink9.onlink.net    |
|drjo006a168.embratel.net.br |
|grail606.nando.net          |
|dd04-016.compuserve.com     |
|134.157.11.7                |
+----------------------------+
only showing top 20 rows



In [18]:
paths_df = (logs_df
            .groupBy('path')
            .count()
            .sort('count',ascending=False))
paths_counts = (paths_df
                .select('path', 'count')
                .rdd.map(lambda r:(r[0],r[1]))
                .collect())
paths, counts = zip(*paths_counts)

In [19]:
print('Top Ten Paths:')
paths_df.show(truncate=False, n=10)

Top Ten Paths:
+----------------------------+------+
|path                        |count |
+----------------------------+------+
|/images/NASA-logosmall.gif  |111087|
|/images/KSC-logosmall.gif   |89530 |
|/images/MOSAIC-logosmall.gif|60300 |
|/images/USA-logosmall.gif   |59845 |
|/images/WORLD-logosmall.gif |59325 |
|/images/ksclogo-medium.gif  |58616 |
|/images/launch-logo.gif     |40841 |
|/shuttle/countdown/         |40251 |
|/ksc.html                   |40067 |
|/images/ksclogosmall.gif    |33555 |
+----------------------------+------+
only showing top 10 rows



In [20]:
from pyspark.sql.functions import desc
not200DF = logs_df.filter(col('status')!=200)
not200DF.show(n=10)
logs_sum_df = (not200DF
               .groupBy('path')
               .count()
               .orderBy(col('count').desc()))
print('Top 10 failed URLs: ')
logs_sum_df.show(truncate=False, n=10)

+--------------------+--------------------+------+------------+--------------------+
|                host|                path|status|content_size|                time|
+--------------------+--------------------+------+------------+--------------------+
| burger.letters.com |/shuttle/countdow...|   304|           0|1995-07-01 00:00:...|
| burger.letters.com |/images/NASA-logo...|   304|           0|1995-07-01 00:00:...|
|     129.94.144.152 |/images/ksclogo-m...|   304|           0|1995-07-01 00:00:...|
|     205.189.154.54 |/cgi-bin/imagemap...|   302|         110|1995-07-01 00:01:...|
|dd15-062.compuser...|/news/sci.space.s...|   404|           0|1995-07-01 00:01:...|
| onyx.southwind.net |/images/KSC-logos...|   304|           0|1995-07-01 00:01:...|
|remote27.compusma...|/cgi-bin/imagemap...|   302|         110|1995-07-01 00:01:...|
|  netport-27.iu.net |/images/ksclogo-m...|   304|           0|1995-07-01 00:02:...|
|  netport-27.iu.net |/images/NASA-logo...|   304|           0|19

In [21]:
from pyspark.sql.functions import countDistinct
unique_host_count = logs_df.agg(countDistinct(col('host'))).head()[0]
print('Unique hosts: {0}'.format(unique_host_count))

Unique hosts: 81982


In [22]:
from pyspark.sql.functions import dayofmonth
day_to_host_pair_df = (logs_df
                       .select(col('host'), dayofmonth(col('time'))
                       .alias('day')))
day_group_hosts_df = day_to_host_pair_df.distinct()
daily_hosts_df = (day_group_hosts_df
                  .groupBy(col('day'))
                  .count()
                  .orderBy('day')
                 )
print('Unique hosts per day:')
daily_hosts_df.show(truncate=False, n=30)

Unique hosts per day:
+---+-----+
|day|count|
+---+-----+
|1  |5192 |
|2  |4859 |
|3  |7336 |
|4  |5524 |
|5  |7383 |
|6  |7820 |
|7  |6474 |
|8  |2898 |
|9  |2554 |
|10 |4464 |
|11 |4927 |
|12 |5345 |
|13 |6951 |
|14 |5297 |
|15 |3116 |
|16 |3013 |
|17 |4943 |
|18 |4523 |
|19 |4919 |
|20 |4729 |
|21 |4339 |
|22 |2575 |
|23 |2635 |
|24 |4298 |
|25 |4376 |
|26 |4296 |
|27 |4368 |
|28 |2175 |
+---+-----+



In [23]:
rows = daily_hosts_df.collect()
days_with_hosts = daily_hosts_df.rdd.map(lambda r: r[0]).collect()
hosts = daily_hosts_df.rdd.map(lambda r:r[1]).collect()
print(days_with_hosts)
print(hosts)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
[5192, 4859, 7336, 5524, 7383, 7820, 6474, 2898, 2554, 4464, 4927, 5345, 6951, 5297, 3116, 3013, 4943, 4523, 4919, 4729, 4339, 2575, 2635, 4298, 4376, 4296, 4368, 2175]


In [25]:
total_req_per_day_df = (logs_df
                        .groupBy(dayofmonth(col('time')).alias('day'))
                        .count())
avg_daily_req_per_host_df = (total_req_per_day_df
                             .join(daily_hosts_df, daily_hosts_df['day']==total_req_per_day_df['day'],'outer')
                             .select(daily_hosts_df['day'],(total_req_per_day_df['count']/daily_hosts_df['count']).alias('avg_reqs_per_host_per_day')))
print('Average number of daily requests per Hosts is :')
avg_daily_req_per_host_df.sort('day').show(n=30)
days_with_avg = avg_daily_req_per_host_df.rdd.map(lambda r:r[0]).collect()
avgs = avg_daily_req_per_host_df.rdd.map(lambda r:r[1]).collect()

Average number of daily requests per Hosts is :
+---+-------------------------+
|day|avg_reqs_per_host_per_day|
+---+-------------------------+
|  1|       12.464175654853621|
|  2|        12.40275776908829|
|  3|       12.211559432933479|
|  4|       12.753801593048516|
|  5|       12.809833401056482|
|  6|       12.910485933503836|
|  7|       13.474358974358974|
|  8|       13.411663216011043|
|  9|       13.810493343774471|
| 10|        16.32168458781362|
| 11|       16.319667140247613|
| 12|       17.312628624883068|
| 13|       19.307006186160265|
| 14|       15.877477817632622|
| 15|        14.61232349165597|
| 16|       15.882509127115831|
| 17|       15.169128059882663|
| 18|       14.212248507627681|
| 19|       14.787151860134173|
| 20|       14.081835483188835|
| 21|       14.894906660520858|
| 22|       13.695922330097087|
| 23|       14.876280834914612|
| 24|       14.950907398790134|
| 25|        14.32792504570384|
| 26|       13.698556797020483|
| 27|        14.12087912

In [28]:
not_found_df = logs_df.filter(col('status')==404)
print('Found {0} 404 URLs'.format(not_found_df.count()))

Found 10845 404 URLs


In [30]:
not_found_paths_df = not_found_df.select('path')
unique_not_found_paths_df  = not_found_paths_df.distinct()
print('404 URLs: ')
unique_not_found_paths_df.show(truncate=False, n=40)

404 URLs: 
+-----------------------------------------------------------------+
|path                                                             |
+-----------------------------------------------------------------+
|/history/apollo/a-001/news/                                      |
|/shuttle/missions/sts-68/images/images.html                      |
|/history/apollo/apollo-13/apollo-13.html)                        |
|/shuttle/sts-71/movies                                           |
|/shuttle/missions/sts-77/ht                                      |
|/htbin/freq                                                      |
|/Harvest/brokers/WWW/stats.html                                  |
|/home/guided-tour.html                                           |
|/shuttle/technology/sts-newsref/sts_sys.html#sts-flag            |
|/history/apollo/a-003/movies/                                    |
|/shuttle/technology/sts-newsref/sts_overview.ht                  |
|/local                              

In [31]:
top_20_not_found_df = (not_found_paths_df
                       .groupBy('path')
                       .count()
                       .orderBy(col('count').desc()))
print('Top 20 404 URLs:')
top_20_not_found_df.show(n=20, truncate=False)

Top 20 404 URLs:
+-----------------------------------------------------------------+-----+
|path                                                             |count|
+-----------------------------------------------------------------+-----+
|/pub/winvn/readme.txt                                            |667  |
|/pub/winvn/release.txt                                           |547  |
|/history/apollo/apollo-13.html                                   |286  |
|/history/apollo/a-001/a-001-patch-small.gif                      |230  |
|/shuttle/resources/orbiters/atlantis.gif                         |230  |
|/://spacelink.msfc.nasa.gov                                      |215  |
|/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif|215  |
|/images/crawlerway-logo.gif                                      |214  |
|/history/apollo/sa-1/sa-1-patch-small.gif                        |183  |
|/shuttle/resources/orbiters/discovery.gif                        |180  |
|/shuttle/missions/st

In [32]:
hosts_404_count_df = (not_found_df
                      .groupBy('host')
                      .count()
                      .orderBy(col('count').desc()))
print('Top 25 hosts that generated errors')
hosts_404_count_df.show(n=25, truncate=False)

Top 25 hosts that generated errors
+----------------------------+-----+
|host                        |count|
+----------------------------+-----+
|hoohoo.ncsa.uiuc.edu        |251  |
|jbiagioni.npt.nuwc.navy.mil |131  |
|piweba3y.prodigy.com        |110  |
|piweba1y.prodigy.com        |92   |
|phaelon.ksc.nasa.gov        |64   |
|www-d4.proxy.aol.com        |61   |
|monarch.eng.buffalo.edu     |56   |
|piweba4y.prodigy.com        |56   |
|alyssa.prodigy.com          |54   |
|titan02f                    |53   |
|www-a2.proxy.aol.com        |52   |
|www-b4.proxy.aol.com        |48   |
|www-b6.proxy.aol.com        |44   |
|www-b3.proxy.aol.com        |43   |
|tearnest2.stpaul.ncr.com    |42   |
|www-d1.proxy.aol.com        |41   |
|www-b2.proxy.aol.com        |41   |
|piweba2y.prodigy.com        |38   |
|www-d3.proxy.aol.com        |38   |
|www-a1.proxy.aol.com        |38   |
|scooter.pa-x.dec.com        |34   |
|proxy.austin.ibm.com        |34   |
|www-d2.proxy.aol.com        |33   |
|ww

In [36]:
errors_by_date_sorted_df = (not_found_df
                            .groupBy(dayofmonth('time').alias('day'))
                            .count()
                            .orderBy('day'))
print('404 errors by day:')
errors_by_date_sorted_df.show(n=30)
days_with_errors_404 = errors_by_date_sorted_df.rdd.map(lambda r:r[0]).collect()
errors_404_by_day = errors_by_date_sorted_df.rdd.map(lambda r:r[1]).collect()
print(days_with_errors_404)
print(errors_404_by_day)

404 errors by day:
+---+-----+
|day|count|
+---+-----+
|  1|  316|
|  2|  291|
|  3|  474|
|  4|  359|
|  5|  497|
|  6|  640|
|  7|  570|
|  8|  302|
|  9|  348|
| 10|  398|
| 11|  471|
| 12|  471|
| 13|  532|
| 14|  413|
| 15|  254|
| 16|  257|
| 17|  406|
| 18|  465|
| 19|  639|
| 20|  428|
| 21|  334|
| 22|  192|
| 23|  233|
| 24|  328|
| 25|  461|
| 26|  336|
| 27|  336|
| 28|   94|
+---+-----+

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
[316, 291, 474, 359, 497, 640, 570, 302, 348, 398, 471, 471, 532, 413, 254, 257, 406, 465, 639, 428, 334, 192, 233, 328, 461, 336, 336, 94]


In [37]:
top_err_date_df = errors_by_date_sorted_df.orderBy(col('count').desc())
print('Top five dates for 404 requests')
top_err_date_df.show(5)

Top five dates for 404 requests
+---+-----+
|day|count|
+---+-----+
|  6|  640|
| 19|  639|
|  7|  570|
| 13|  532|
|  5|  497|
+---+-----+
only showing top 5 rows



In [46]:
from pyspark.sql.functions import hour
hour_records_sorted_df = (not_found_df
                          .groupBy(hour(col('time')).alias('hour'))
                          .count()
                          .orderBy('hour'))
print('Top hours for 404 requests')
hour_records_sorted_df.show(24)
hours_with_not_found = hour_records_sorted_df.rdd.map(lambda r:r[0]).collect()
not_found_counts_per_hour = hour_records_sorted_df.rdd.map(lambda r:r[1]).collect()
print(hours_with_not_found)
print(not_found_counts_per_hour)

Top hours for 404 requests
+----+-----+
|hour|count|
+----+-----+
|   0|  430|
|   1|  321|
|   2|  268|
|   3|  240|
|   4|  168|
|   5|  147|
|   6|  134|
|   7|  240|
|   8|  365|
|   9|  481|
|  10|  595|
|  11|  732|
|  12|  657|
|  13|  537|
|  14|  752|
|  15|  833|
|  16|  631|
|  17|  617|
|  18|  505|
|  19|  412|
|  20|  383|
|  21|  445|
|  22|  485|
|  23|  467|
+----+-----+

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[430, 321, 268, 240, 168, 147, 134, 240, 365, 481, 595, 732, 657, 537, 752, 833, 631, 617, 505, 412, 383, 445, 485, 467]
